# COGS 118A- Project Proposal

# Project Description

You will design and execute a machine learning project. There are a few constraints on the nature of the allowed project. 
- The problem addressed will not be a "toy problem" or "common training students problem" like mtcars, iris, palmer penguins etc.
- The dataset will have >1k observations and >5 variables. I'd prefer more like >10k observations and >10 variables. A general rule is that if you have >100x more observations than variables, your solution will likely generalize a lot better. The goal of training a supervised machine learning model is to learn the underlying pattern in a dataset in order to generalize well to unseen data, so choosing a large dataset is very important.

- The project will include a model selection and/or feature selection component where you will be looking for the best setup to maximize the performance of your ML system.
- You will evaluate the performance of your ML system using more than one appropriate metric
- You will be writing a report describing and discussing these accomplishments


Feel free to delete this description section when you hand in your proposal.

### Peer Review

You will all have an opportunity to look at the Project Proposals of other groups to fuel your creativity and get more ideas for how you can improve your own projects. 

Both the project proposal and project checkpoint will have peer review.

# Names

- Xiaolong Zhou
- Yiling Cao
- Yuqin Zhang

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- In addressing the need for automated music genre identification for enhanced user experience on music streaming platforms, we propose a machine learning approach to tackle genre classification and personalized music recommendation.
- Leveraging data features like tempo, rhythm, pitch, and lyrical content, our goal is to predict genre labels, a complex multiclass classification task that emulates the human ability to discern music genres. Concurrently, we aim to rank songs based on user preferences, transforming personalized music recommendation into an information retrieval task. Data used in this project will represent a variety of music genres and user listening habits, measured through both objective attributes of music and subjective user interactions.
- Machine learning algorithms will be utilized to make sense of this data, mapping musical features to genre labels and user preferences to song rankings.
- The performance of our genre prediction model will be gauged using standard classification metrics, such as accuracy, precision, recall, and F1-score, ensuring a quantitative measure of our success in mimicking the sensitive human ability to identify sound.

# Background
Music genre classification is a significant area of interest in digital music processing. The process involves extracting acoustic features of music, such as genre, rhythmic structure, harmonic content, duration, and the instruments played, using digital signal processing techniques. The classification plays a critical role in recommendation systems, audio libraries organization, and trend discovery<a name="ieee"></a>[<sup>[1]</sup>](#ieee)​.

Musical genres are human-made categories, defining different types of music based on shared features, including instrumentation, rhythmic structure, and harmonic content. These genre hierarchies help in managing the enormous volume of music available on the internet<a name="ssrn"></a>[<sup>[2]</sup>](#ssrn)​.

Various approaches, including Multiclass support vector machine, K-Nearest Neighbors, K-means clustering algorithm, and Convolutional neural network, are utilized in building music genre classification systems. The use of deep learning models, particularly convolutional neural networks, has gained popularity in automatic music genre classification, with a comparative study showing promising results against traditional classifiers<a name="raghav"></a>[<sup>[3]</sup>](#raghav).

Music recommendation systems are another crucial aspect of digital music platforms. These systems predict and push user preferences based on user behavior and music characteristics. The development of these algorithms has shifted from merely focusing on individual user preferences to incorporating mutual recommendations among users<a name="hindawi"></a>[<sup>[4]</sup>](#hindawi).

In summary, music genre identification and recommendation systems serve as essential components of digital music platforms, enhancing user experience by organizing music content and tailoring music selections to individual preferences. These systems leverage machine learning and deep learning methods to analyze music characteristics and user behavior, thereby enabling accurate genre classification and personalized music recommendations.

# Problem Statement

Human sound identify abbility is usually very sensitive. People usually identify a music genre "based on both objective and subjective measures"<a name="Lippens"></a>[<sup>[5]</sup>](#Lippens). When human subjectives is not possible to duplicate in machines, many music platforms still want machines to be able to identify such music genre as a source of feature preference. Music streaming platforms face two key challenges: accurate genre classification and personalized music recommendation. We propose to address these problems using machine learning. The genre prediction problem is a multiclass classification task, mapping input features (tempo, rhythm, pitch, lyrical content) to predefined genre labels. The recommendation problem can be defined as ranking songs based on user preferences, expressed as an information retrieval task. The problems can be mathematically defined and solved using ML algorithms. The genre prediction’s performance can be assessed using accuracy, precision, recall, and F1-score.

# Data

We are going to use a dataset from Kaggle [(Link)](https://www.kaggle.com/datasets/vicsuperman/prediction-of-music-genre?select=music_genre.csv) that describes music genres: 'Electronic', 'Anime', 'Jazz', 'Alternative', 'Country', 'Rap', 'Blues', 'Rock', 'Classical', 'Hip-Hop'. This dataset is about 3 MB, with 50006 observations and 18 variables: instance_id, arist_name, track_name, popularity, acousticness, dancebnility, duration_ms, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, obtained_date, valence, and music_genre. 

In the dataset, we will observe different genre of music with the digitized features of a music. We observed that for example, that many tracks of "Classical" music has a very high value of acousticness. "Hip-Hop", "Rap", and "Rock" musics have higher popularity values compare to other genres. We think that all the music element related variables are critical for such genre predictions. For example, tempo will be a critical variable to identify if a music is fast or not. And we will rarely see such fast tempo music in "Blues" or "Jazz". 

Base on the fact that most of the variables are being numerical values, we do not need much of data transformations to do. We might eliminate some of the variables that shows less importance after some analysis of the data and do some standardization to balanced out all of the variable values. 

In [2]:
# import pandas & numpy library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setup plotting in a notebook in a reasonable way
%matplotlib inline
%config InlineBackend.figure_format='retina'

# default figure aesthetics I'll be using, 
# there are other choices, see seaborn docs
sns.set_style("darkgrid")
sns.set_context("notebook")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import plot_tree
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [4]:
path = "/Users/zhangyuqin/Desktop/music_genre.csv"
music_data = pd.read_csv(path)

# Drop unrelated columns
music_data = music_data.drop(['artist_name', 'track_name', 'obtained_date'], axis=1)

# Replace missing values
music_data.replace(['?', -1], np.nan, inplace=True)

# Convert data types
music_data['duration_ms'] = music_data['duration_ms'].astype(float)
music_data['tempo'] = music_data['tempo'].astype(float)

# Standardize text data
music_data['key'] = music_data['key'].str.lower().str.strip()
music_data['mode'] = music_data['mode'].str.lower().str.strip()
music_data['music_genre'] = music_data['music_genre'].str.lower().str.strip()

# Handle NaN values (this is just one approach, you'll need to decide what's best for your use case)
music_data.dropna(inplace=True)
music_data['music_genre'] = np.where(music_data['music_genre'] != 'classical', 'non-classical', music_data['music_genre'])
print(music_data.head())

   instance_id  popularity  acousticness  danceability  duration_ms  energy  \
1      46652.0        31.0       0.01270         0.622     218293.0   0.890   
2      30097.0        28.0       0.00306         0.620     215613.0   0.755   
3      62177.0        34.0       0.02540         0.774     166875.0   0.700   
4      24907.0        32.0       0.00465         0.638     222369.0   0.587   
6      43760.0        46.0       0.02890         0.572     214408.0   0.803   

   instrumentalness key  liveness  loudness   mode  speechiness    tempo  \
1          0.950000   d     0.124    -7.043  minor       0.0300  115.002   
2          0.011800  g#     0.534    -4.617  major       0.0345  127.994   
3          0.002530  c#     0.157    -4.498  major       0.2390  128.014   
4          0.909000  f#     0.157    -6.266  major       0.0413  145.036   
6          0.000008   b     0.106    -4.294  major       0.3510  149.995   

   valence    music_genre  
1    0.531  non-classical  
2    0.333  

In [7]:
# Standardize numerical columns
numerical_cols = ['popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']
music_data[numerical_cols] = (music_data[numerical_cols] - music_data[numerical_cols].mean()) / music_data[numerical_cols].std()

# Perform one-hot encoding on the 'key' and 'mode' columns
df = pd.get_dummies(music_data, columns=['key', 'mode'])

# Exclude the 'music_genre' column from the feature selection
X = df.drop('music_genre', axis=1)
y = df['music_genre']

# Perform feature selection to select the top k features
k = 5  # Number of features to select
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)

# Get the selected feature names
selected_feature_names = X.columns[selector.get_support(indices=True)].tolist()

model = DecisionTreeClassifier()
# Split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Tuning
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)

# Train a decision tree with the best parameters
dtc_best = DecisionTreeClassifier(**grid_search.best_params_)
dtc_best.fit(X_train, y_train)
y_pred_best = dtc_best.predict(X_test)
print(classification_report(y_test, y_pred_best))

               precision    recall  f1-score   support

    classical       0.76      0.73      0.75       821
non-classical       0.97      0.97      0.97      7291

     accuracy                           0.95      8112
    macro avg       0.86      0.85      0.86      8112
 weighted avg       0.95      0.95      0.95      8112

Best parameters:  {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 5}
               precision    recall  f1-score   support

    classical       0.85      0.73      0.79       821
non-classical       0.97      0.99      0.98      7291

     accuracy                           0.96      8112
    macro avg       0.91      0.86      0.88      8112
 weighted avg       0.96      0.96      0.96      8112



In [8]:
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Create a SVM classifier
clf = svm.SVC(kernel='linear')  # Linear Kernel
# Train the model using the training sets
clf.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

# Define the parameter grid
param_grid = {
    'C': [10,100],  # Regularization parameter
    'gamma': [0.01, 0.001],  # Kernel coefficient
    'kernel': ['rbf']  # Type of SVM
}

# Create the GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters: ", grid_search.best_params_)

# Print the best score
print("Best score: ", grid_search.best_score_)

# Predict the response for test dataset using the best model
y_pred = grid_search.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

    classical       0.90      0.08      0.14       821
non-classical       0.91      1.00      0.95      7291

     accuracy                           0.91      8112
    macro avg       0.90      0.54      0.55      8112
 weighted avg       0.91      0.91      0.87      8112



# Proposed Solution

By generating multiple music genre classification results using models such as neural network, KNN, and decision trees, we will maintain an accuracy percentage of our model to show how efficient such model can be with only subjective features we gained from the data. We have also tried implement sentimental data values from the dataset to try relate as close as possible to the true genre. These sentimental features are included as: popularity, acousticness, energy, intrumental, liveness, loudness, speechiness, tempo, valence. We might exclude "artist_name" and "track_name" since we want to focus on the element of a music genre as music it self instead of its title and author. For features such as key and mode, we will be doing one-hot-encoding to transform these categorical data to the same type as other features. As human, we are able to identify if such music is lively or not and base off that detect its genre. For this model, we want to simulate the same detection that our human can do toward the musics itself. We will be spliting our 50006 observations into train, valid, and test datasets. Using methods such as nested cross-validation would gave us accurate results from multiple different algorithm based models. We will be mainly observe a KNN and decision tree model by comparing and contrast each runtime and accuracy. We migh also need to do one-hot encoding since our labels are multiclass categorical. By tuning the parameters, we will find the best number of neighbors for KNN and best fitted tree depth and criterion for decision tree model. Idealy, a neural net model should be more accurate and widely used. However, with limited time and knowledge on neural networks, our team plan on test such algorithm model after we have a finite solution first. Our solution model can help our main project problem by providing a source to music platforms to quickly identify a genre when music data are being imported to the platform. It will also be helpful when the music platform tries to build a recommendation system and apply such model to help label what type of music is each user listening to. 

There is a benchmark model called "music_genre_classification" using Convolutional neural network(CNN) to perform music genre classification. [Link](https://github.com/ds7711/music_genre_classification/tree/master). This model has achieved an accuracy of 70%, which is comparable to the human accuracy. Such model can possiblly substitute human ears and identify music genres within 30 seconds of the music play. It will be more time saving and allow humans to apply their abilities on other fields with more time.

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

A mean accuracy from the nested cross validation would be the most desirable evaluation metrics that we will be using in this project. When we are doing a classification problem, we want to know how good is our model doing. In the benchmark model, the author provided their accuracy percentage on the test dataset. Our model first will classify test dataset music genres with the features only. Then we will take those labels accompany with the obervation ids to find whether or not the true label is the same as predicted label. Last, we will count up all the correct number of predicted labels and calculate the accuracy. The accuracy can be calculated using below formula: <br>
$
\begin{align}
x &= (x_1, ..., x_m), x_i \in \mathcal{R}^n, x \in \mathcal{R}^m \\
y & \in \mathcal{R}^{10}: \text{['Electronic', 'Anime', 'Jazz', 'Alternative', 'Country', 'Rap', 'Blues', 'Rock', 'Classical', 'Hip-Hop']} \\
accuracy &= 1 - error \\
error &= \frac{1}{n}\sum^n_{i=1}1(y_i \neq f(x_i))\\
\end{align}$

Since we planning on using nested cross-validation, each iteration will contain a best accuracy. In order to compare algorithms to algorithms, for example KNN to decision tree, we have to average such accuracy in total iterations and compare them. So, we might need to do multiple cross-validations to ensure our model is working efficiently. 

# Ethics & Privacy

Data Permission & Privacy:
 The dataset: “Prediction of Music Genre” that we select is chosen from a public database website: Kaggle. Unlike other datasets that focus on human study, our chosen data collects only the description (artist name and track name, nothing on personal identities or ethnic background) and quantified structural information (popularity, duration, energy, key, etc.) of a variety of songs that are already posted online (collected through Spotify API); With the fact that it does not actually contain any of the song file, we eliminate all issues with regard to copyright and user privacy. Additionally, this particular dataset follows the CC0 1.0 Universal Public Domain Dedication, thus we have the right to use, modify and perform work on it. \

Potential Bias: 
 It is essential to notice that music genres are affected by various of aspects. Due to the limitation of variables in this dataset, we might not capture all the differences between genres to fully distinguish one from another and discover a feature bias. \
 
Implication:
 Since this model will only be used in this particular in-class project, we will interpret the result of our model in caution as for now, and we may keep researching and modifying the variables we use.


# Team Expectations 

* We expect each member to contribute equally during group discussion.
* We expect to conduct a thoughtful and sophisticated analysis to our data that leads to the best solution.
* We should respect each other’s idea.
* We expect to be unanimous on what we try to accomplish.
* We will thoroughly fulfill the timeline proposal.

# Project Timeline Proposal

Replace this with something meaningful that is appropriate for your needs. It doesn't have to be something that fits this format.  It doesn't have to be set in stone... "no battle plan survives contact with the enemy". But you need a battle plan nonetheless, and you need to keep it updated so you understand what you are trying to accomplish, who's responsible for what, and what the expected due dates are for each item.

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 5.10  |  8 PM |  Read project instructions; brainstorm for topics  | Draft a timeline that works for everyone; decide topic | 
| 5.14  |  10 AM |  Do background research for topic | Discuss and pick a dataset related to music genre; draft project proposal | 
| 5.17  | 10 AM  | Finalize and submit project proposal | Assign group members to each specific part; discuss approach to solution  |
| 5.27  | 10 AM  | Data wrangling, analysis; Model training | Be ready to submit project checkpoint   |
| 6.9   | 8 PM  | Model Testing, selecting; Complete Analysis | Draft results; draft final project file |
| 6.14  | Before 11:59 PM  | Finish final project| Submit final project |

# Footnotes
<a name="ieee"></a>1.[^](#ieee):Music Genre Classification and Recommendation by Using ... - IEEE Xplore, ieeexplore.ieee.org/document/8554016. Accessed 17 May 2023. https://ieeexplore.ieee.org/document/8554016<br> 
<a name="ssrn"></a>2.[^](#ssrn): Dias, Jessica, et al. “Music Genre Classification &amp; Recommendation System Using CNN.” SSRN, 18 May 2022, papers.ssrn.com/sol3/papers.cfm?abstract_id=4111849.https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4111849 <br>
<a name="raghav"></a>3.[^](#raghav): Agrawal, Raghav. “Music Genre Classification Project Using Machine Learning Techniques.” Analytics Vidhya, 6 Apr. 2022, https://www.analyticsvidhya.com/blog/2022/03/music-genre-classification-project-using-machine-learning-techniques/.  <br>
<a name="hindawi"></a>4.[^](#hindawi): Zhang, Yezi. “Music Recommendation System and Recommendation Model Based on Convolutional Neural Network.” Mobile Information Systems, 12 May 2022, https://www.hindawi.com/journals/misy/2022/3387598/.  <br>
<a name="Lippens"></a>5.[^](#Lippens): Lippens, S., Martens, J.P, Leman, M., Baets, B., Meyer, H. A COMPARISON OF HUMAN AND AUTOMATIC MUSICAL GENRE CLASSIFICATION. *ResearchGate*. https://www.researchgate.net/publication/200806218_A_Comparison_of_Human_and_Automatic_Musical_Genre_Classification<br> 
